In [ ]:
!(@isdefined cleanup_hook) && IJulia.push_postexecute_hook(() -> (empty!(In); empty!(Out); GC.gc(true))); cleanup_hook = true;

In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")
include("../../../functions/utils.jl")

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure9/clustering"
mkpath(fig_path)

In [ ]:
data_info_dict = load("/home/chuyu/Notebooks/project_place_cell/figures/chuyu/figure_data_info.jld2")

data_info_all = []
data_info_tog = []
job_names = []
for key in keys(data_info_dict)
    append!(data_info_all, data_info_dict[key])
    append!(data_info_tog, [data_info_dict[key]])
    append!(job_names, [String(key)])
end

In [ ]:
analyzer = "chuyu"

# Example anatomy

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure3/clustering_anatomy"
mkpath(fig_path)

In [ ]:
data_info_all = data_info_dict["corner_cue_rotation_whole"]

which_data = 3
data_info = data_info_all[which_data]





experiment_filename_1 = data_info[1]
server_1 = data_info[2]

experiment_filename_2 = data_info[3]
server_2 = data_info[4]

experimenter = data_info[5]

ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")



NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
global Z_all = HDF5.readmmap(NMF_file["Z_all"])
global X_all = HDF5.readmmap(NMF_file["X_all"])
global Y_all = HDF5.readmmap(NMF_file["Y_all"])
global A_dFF_1 = HDF5.readmmap(NMF_file["A_dFF"]); # bs update
close(NMF_file)



save_file_name = "compare_map_results_original.h5"
compare_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(compare_filename, "r")
tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
confined_place_cell_index_1 = HDF5.readmmap(file["confined_place_cell_index_1"])
confined_place_cell_index_2 = HDF5.readmmap(file["confined_place_cell_index_2"])
peak_loc_map_1_precise_array = HDF5.readmmap(file["peak_loc_map_1_precise"])
peak_loc_map_2_precise_array = HDF5.readmmap(file["peak_loc_map_2_precise"])
close(file)




In [ ]:
data_info

In [ ]:
@pyimport matplotlib.cm as cm
@pyimport sklearn.metrics as metrics
@pyimport colorsys
@pyimport cmocean

# hsv_cmap = cm.get_cmap("hsv", 360)
hsv_cmap = cmocean.cm.phase

In [ ]:
function hsv_control(rgb_color, s, v)
    color_hsv = colorsys.rgb_to_hsv(rgb_color[1],rgb_color[2],rgb_color[3])
    color_rgb = colorsys.hsv_to_rgb(color_hsv[1],s,v)
    return color_rgb
end

In [ ]:
# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_1), "chamber_geometry_$(experiment_filename_1).h5"))
chamber_roi_1 = read(orientation_correction_file,"chamber_roi")
countour_array_1 = read(orientation_correction_file,"countour_array")
center_loc_1 = read(orientation_correction_file,"center_loc")
close(orientation_correction_file)

In [ ]:
distance_center_1 = distance_from(collect(peak_loc_map_1_precise_array'), center_loc_1);
distance_edge_1, _ = distance_to_feature(collect(peak_loc_map_1_precise_array'), countour_array_1);

angle_center_1 = angle_from(peak_loc_map_1_precise_array', center_loc_1);
angle_center_1[angle_center_1.<0]=angle_center_1[angle_center_1.<0].+2*pi;

In [ ]:
rgb_color = hsv_cmap(Int32.(round.(180*angle_center_1/pi)))
hsv_s = [(maximum(distance_edge_1) .-distance_edge_1[i])/maximum(distance_edge_1)+1 for i in 1:size(rgb_color, 1)];
hsv_v = [(maximum(distance_edge_1) .-distance_edge_1[i])/maximum(distance_edge_1)+0.3 for i in 1:size(rgb_color, 1)];
hsv_s[hsv_s.!=1].=1
hsv_v[hsv_v.>1].=1
rgb_color_s = [hsv_control(rgb_color[i,:], hsv_s[i],hsv_v[i]) for i in 1:size(rgb_color, 1)];

In [ ]:
scatter(peak_loc_map_1_precise_array[1,:], peak_loc_map_1_precise_array[2,:], c= rgb_color_s)

In [ ]:
    NMF_filename = joinpath(data_path(ds_save_1), "NMF.h5")
    NMF_file = h5open(NMF_filename, "r")
    try
        global ds_stack_mean = copy(HDF5.readmmap(NMF_file["stack_mean"]))
    catch
        global ds_stack_mean = copy(HDF5.readmmap(NMF_file["ds_stack_mean"]))
    end


    close(NMF_file)

In [ ]:
fig_path

In [ ]:
stack_img = ds_stack_mean


which_neuron = confined_place_cell_index_1

fig, ax = subplots(1,1,dpi=250, figsize=(8.2, 4.6))
fig_bg = maximum(stack_img, dims=3)[:,:,1]
ax.imshow(fig_bg', cmap="gray")
ax.scatter(X_all[which_neuron], Y_all[which_neuron], c=rgb_color_s, s=10)
axis("off")
xlim(550, size(stack_img,1))
ylim(100, 350)
fig.savefig(joinpath(fig_path, "anatomy_distribution_1.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);


fig, ax = subplots(1,1,dpi=250, figsize=(8.2, 1.5))
fig_bg = maximum(stack_img, dims=2)[:,1,:]
ax.imshow(fig_bg', cmap="gray",aspect=2)
ax.scatter(X_all[which_neuron], Z_all[which_neuron], c=rgb_color_s, s=2)
axis("off")
xlim(550, size(stack_img,1))
ylim(100, 25)
fig.savefig(joinpath(fig_path, "anatomy_distribution_2.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);


fig, ax = subplots(1,1,dpi=250, figsize=(4.6, 1.5))
fig_bg = maximum(stack_img, dims=1)[1,:,:]
ax.imshow(fig_bg', cmap="gray",aspect=2)
ax.scatter(Y_all[which_neuron], Z_all[which_neuron], c=rgb_color_s, s=2)
axis("off")
xlim(100, 350)
ylim(100, 25)
fig.savefig(joinpath(fig_path, "anatomy_distribution_3.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);
